In [42]:
import findspark
findspark.init()


In [43]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, when

In [34]:
spark = SparkSession.builder \
    .appName("Read CSV with Header and Delimiter") \
    .getOrCreate()

In [35]:
pricing_path='./pricing_project_dataset.csv'


In [36]:
df = spark.read \
    .option("header", "true") \
    .option("delimiter", ",") \
    .csv(pricing_path)

df.show()


+------------+--------------+--------------+--------------+---------------+------------------+-------------------+----------------------+------------------+----------------------+--------------+------------+-----------------------------------+
|grass_region|category_group|   seller_type|shopee_item_id|shopee_model_id|competitor_item_id|competitor_model_id|shopee_model_price_usd|shopee_model_price|competitor_model_price|shopee_gmv_usd|shopee_order|shopee_model_competitiveness_status|
+------------+--------------+--------------+--------------+---------------+------------------+-------------------+----------------------+------------------+----------------------+--------------+------------+-----------------------------------+
|          ID|   Electronics|Long Tail (LT)|    5646734211|     3953474697|        9153508336|          992466002|           50.33644918|            779233|                647903|   8758.537644|         174|                       Shopee > CPT|
|          SG|          

In [37]:
condition_1 = df.filter(df['shopee_model_competitiveness_status'] == 'Shopee < CPT') \
                .select('shopee_item_id') \
                .distinct() \
                .withColumn('item_competitiveness', when(col('shopee_item_id').isNotNull(), 'Shopee < CPT'))


In [38]:
condition_2 = df.filter(df['shopee_model_competitiveness_status'] == 'Shopee = CPT') \
                .select('shopee_item_id') \
                .distinct() \
                .withColumn('item_competitiveness', when(col('shopee_item_id').isNotNull(), 'Shopee = CPT'))


In [39]:
condition_3 = df.filter(df['shopee_model_competitiveness_status'] == 'Shopee > CPT') \
                .select('shopee_item_id') \
                .distinct() \
                .withColumn('item_competitiveness', when(col('shopee_item_id').isNotNull(), 'Shopee > CPT'))


In [40]:
others = df.select('shopee_item_id') \
            .distinct() \
            .join(condition_1, 'shopee_item_id', 'left_anti') \
            .join(condition_2, 'shopee_item_id', 'left_anti') \
            .join(condition_3, 'shopee_item_id', 'left_anti') \
            .withColumn('shopee_model_competitiveness', when(col('shopee_item_id').isNotNull(), 'Others'))

In [41]:
result = condition_1.union(condition_2).union(condition_3).union(others)

result.show()

+--------------+--------------------+
|shopee_item_id|item_competitiveness|
+--------------+--------------------+
|    8027930079|        Shopee < CPT|
|    4427653601|        Shopee < CPT|
|    2824466205|        Shopee < CPT|
|    5341122547|        Shopee < CPT|
|    4462310061|        Shopee < CPT|
|    6357434787|        Shopee < CPT|
|    4587193987|        Shopee < CPT|
|    5606773200|        Shopee < CPT|
|    5624048760|        Shopee < CPT|
|    2476005413|        Shopee < CPT|
|    2973248760|        Shopee < CPT|
|    1623598857|        Shopee < CPT|
|    6246545229|        Shopee < CPT|
|    9931391521|        Shopee < CPT|
|    2378507297|        Shopee < CPT|
|    7943458464|        Shopee < CPT|
|    9491284060|        Shopee < CPT|
|    2310826981|        Shopee < CPT|
|    5776945463|        Shopee < CPT|
|    8882022439|        Shopee < CPT|
+--------------+--------------------+
only showing top 20 rows

